In [2]:
import json
import datetime
import time
import yaml
import pandas as pd
import mojito
import pprint
import requests
import pymongo

In [8]:
with open('C:/Users/cc843/Desktop/辛旻宗/주식프로젝트/key/config.yaml', encoding='UTF-8') as f:
    _cfg = yaml.load(f, Loader=yaml.FullLoader)
APP_KEY = _cfg['APP_KEY']
APP_SECRET = _cfg['APP_SECRET']
ACCESS_TOKEN = ""
CANO = _cfg['CANO']
ACNT_PRDT_CD = _cfg['ACNT_PRDT_CD']
DISCORD_WEBHOOK_URL = _cfg['DISCORD_WEBHOOK_URL']
URL_BASE = _cfg['URL_BASE']

In [11]:
# def get_access_token():
#     """토큰 발급"""
#     headers = {"content-type": "application/json"}
#     body = {"grant_type": "client_credentials", "appkey": APP_KEY, "appsecret": APP_SECRET}
#     PATH = "oauth2/tokenP"
#     URL = f"{URL_BASE}/{PATH}"
#     res = requests.post(URL, headers=headers, data=json.dumps(body))
#     ACCESS_TOKEN = res.json()["access_token"]
#     return ACCESS_TOKEN

In [12]:
# ACCESS_TOKEN = get_access_token()

In [ ]:
broker = mojito.KoreaInvestment(
    api_key=APP_KEY,
    api_secret=APP_SECRET,
    acc_no="50101465-01",
    mock=True
)

resp = broker.fetch_ohlcv(
    symbol="005930",
    timeframe='D',
    adj_price=True,
    #start_day="20200101", 
    end_day="20221230" # end_day부터 앞에 100개의 데이터를 가져옴
)

pprint.pprint(resp)

In [20]:
json = resp
print(json)

{'output1': {'prdy_vrss': '-200', 'prdy_vrss_sign': '5', 'prdy_ctrt': '-0.25', 'stck_prdy_clpr': '78900', 'acml_vol': '9447858', 'acml_tr_pbmn': '747694570500', 'hts_kor_isnm': '삼성전자', 'stck_prpr': '78700', 'stck_shrn_iscd': '005930', 'prdy_vol': '26724760', 'stck_mxpr': '102500', 'stck_llam': '55300', 'stck_oprc': '79600', 'stck_hgpr': '79800', 'stck_lwpr': '78600', 'stck_prdy_oprc': '79600', 'stck_prdy_hgpr': '79900', 'stck_prdy_lwpr': '77800', 'askp': '78800', 'bidp': '78700', 'prdy_vrss_vol': '-17276902', 'vol_tnrt': '0.16', 'stck_fcam': '100', 'lstn_stcn': '5969782550', 'cpfn': '7780', 'hts_avls': '4698219', 'per': '36.93', 'eps': '2131.00', 'pbr': '1.51', 'itewhol_loan_rmnd_ratem name': '0.12'}, 'output2': [{'stck_bsop_date': '20221229', 'stck_clpr': '55300', 'stck_oprc': '56000', 'stck_hgpr': '56200', 'stck_lwpr': '55300', 'acml_vol': '11295935', 'acml_tr_pbmn': '628653753150', 'flng_cls_code': '00', 'prtt_rate': '0.00', 'mod_yn': 'N', 'prdy_vrss_sign': '5', 'prdy_vrss': '-1300'

In [7]:
df = pd.DataFrame(resp['output2'])
dt = pd.to_datetime(df['stck_bsop_date'], format="%Y%m%d")
df.set_index(dt, inplace=True)
df = df[['stck_oprc', 'stck_hgpr', 'stck_lwpr', 'stck_clpr']]
df.columns = ['open', 'high', 'low', 'close']
df.index.name = "date"
df

,open,high,low,close
date,,,,
2022-12-29,56000,56200,55300,55300
2022-12-28,57600,57600,56400,56600
2022-12-27,58000,58400,57900,58100
2022-12-26,58000,58100,57700,57900
2022-12-23,58200,58400,57700,58100
...,...,...,...,...
2022-08-11,59600,60000,59300,59900
2022-08-10,58900,59200,58600,59100
2022-08-09,60600,60700,59600,60000


In [15]:
import mojito
import pprint
from datetime import datetime, timedelta

# 계좌번호 설정
acc_no = "50101465-01"

# 브로커 객체 생성
broker = mojito.KoreaInvestment(
    api_key=APP_KEY,
    api_secret=APP_SECRET,
    acc_no=acc_no
)

# 오늘 날짜 설정
today = datetime.today().strftime("%Y%m%d")

# 2022년 12월 30일부터 시작
start_day = "20221230"
end_day = start_day

# 데이터 리스트
data = []

df = pd.DataFrame(resp['output2'])
dt = pd.to_datetime(df['stck_bsop_date'], format="%Y%m%d")
df.set_index(dt, inplace=True)
df = df[['stck_oprc', 'stck_hgpr', 'stck_lwpr', 'stck_clpr']]
df.columns = ['open', 'high', 'low', 'close']
df.index.name = "date"

# 다음 데이터 추가
while end_day <= today:
    # 100개 일봉 데이터 요청
    resp = broker.fetch_ohlcv(
        symbol="005930",
        timeframe='D',
        adj_price=True,
        end_day=end_day
    )

    # 데이터프레임 생성
    df_temp = pd.DataFrame(resp['output2'])
    dt = pd.to_datetime(df_temp['stck_bsop_date'], format="%Y%m%d")
    df_temp.set_index(dt, inplace=True)
    df_temp = df_temp[['stck_oprc', 'stck_hgpr', 'stck_lwpr', 'stck_clpr']]
    df_temp.columns = ['open', 'high', 'low', 'close']
    df_temp.index.name = "date"

    # 데이터 합치기
    df = pd.concat([df, df_temp])

    # 다음 100일 날짜 설정
    end_day = (datetime.strptime(end_day, "%Y%m%d") + timedelta(days=100)).strftime("%Y%m%d")
    print(end_day)

# 결과 출력
print(df)




20230409
20230718
20231026
20240203
20240513
             open   high    low  close
date                                  
2024-03-22  79600  79900  77800  78900
2024-03-21  79200  79300  77700  79300
2024-03-20  73700  77200  73400  76900
2024-03-19  72300  73000  71700  72800
2024-03-18  72600  73000  72500  72800
...           ...    ...    ...    ...
2023-09-12  70800  71000  70400  70500
2023-09-11  70400  70800  70000  70800
2023-09-08  70200  70300  69600  70300
2023-09-07  70000  70600  69600  70400
2023-09-06  70700  70800  69700  70000

[600 rows x 4 columns]
